# Section 1: Preprocessing

In [ ]:
# Load the original benign dataset locally
data = pd.read_csv("")  # <-- Insert path to ImageNet benign dataset

# Select features:
# Margin loss = Difference between First and Second Logit
# SortedList_Second_Element = Second Logit
# SortedList_first_Element = First Logit
features = ['Margin Loss', 'SortedList_Second_Element', 'SortedList_First_Element']
data_selected = data[features]

# Quick missing value check
print("\nMissing values per feature:\n", data_selected.isnull().sum())

# Scale only 'Margin Loss'
scaler_margin = MinMaxScaler(feature_range=(0, 1))
data_selected.loc[:, 'Margin Loss'] = scaler_margin.fit_transform(
    data_selected[['Margin Loss']]
).ravel().astype('float64')

# Keep original logits (no scaling)
data_selected['SortedList_Second_Element'] = data['SortedList_Second_Element']
data_selected['SortedList_First_Element']  = data['SortedList_First_Element']

data_scaled = pd.DataFrame(data_selected, columns=features)

# Train/validation split for VAE
X_train, X_valid = train_test_split(data_scaled, test_size=0.2, random_state=42)

# Convert to NumPy arrays for the model
X_train = np.array(X_train).astype("float32")
X_valid = np.array(X_valid).astype("float32")
print(f"\nTraining set shape: {X_train.shape}")
print(f"Validation set shape: {X_valid.shape}")

plt.figure(figsize=(12, 8))

# Margin Loss (original vs scaled)
plt.subplot(2, 2, 1)
sns.histplot(data['Margin Loss'], kde=True, bins=30, color="red", alpha=0.6)
plt.title('Original Margin Loss')

plt.subplot(2, 2, 2)
sns.histplot(data_scaled['Margin Loss'], kde=True, bins=30, color="blue", alpha=0.6)
plt.title('Scaled Margin Loss')

# Second Logit
plt.subplot(2, 2, 3)
sns.histplot(data['SortedList_Second_Element'], kde=True, bins=30, color="green", alpha=0.6)
plt.title('Second Logit (Original)')

# First Logit
plt.subplot(2, 2, 4)
sns.histplot(data['SortedList_First_Element'], kde=True, bins=30, color="orange", alpha=0.6)
plt.title('First Logit (Original)')

plt.tight_layout()
plt.show()



# Section 2: Variational Autoencoder Architecture & Training

In [ ]:
latent_dim = 20  # size of the latent space (z)

# Encoder: maps 3 input features -> latent parameters (mean & log-variance)
inputs = Input(shape=(3,), name='encoder_input')  # each row has 3 features
x = Dense(16)(inputs)
x = LeakyReLU(alpha=0.2)(x)
x = Dense(8)(x)
x = LeakyReLU(alpha=0.2)(x)
z_mean = Dense(latent_dim, name='z_mean')(x)       # μ(z)
z_log_var = Dense(latent_dim, name='z_log_var')(x) # log σ^2(z)

# Reparameterization trick: sample z = μ + σ * ε (keeps backprop valid)
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))           # ε ~ N(0, I)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon        # σ = exp(0.5 * log_var)

z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# Decoder: maps latent z back to 3 features (reconstruction)
decoder_input = Input(shape=(latent_dim,), name='decoder_input')
decoder_x = Dense(8)(decoder_input)
decoder_x = LeakyReLU(alpha=0.2)(decoder_x)
decoder_x = Dense(16)(decoder_x)
decoder_x = LeakyReLU(alpha=0.2)(decoder_x)
outputs = Dense(3, activation='tanh', name='decoder_output')(decoder_x)  # 3 outputs to match inputs

decoder = Model(decoder_input, outputs, name='decoder')

# Run decoder on sampled z
outputs = decoder(z)

# Custom VAE Model: adds KL term to the loss (weighted by beta)
class VAE(Model):
    def __init__(self, encoder, decoder, beta=0.1, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.beta = beta  # weight for KL divergence

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)

        # KL divergence term: encourages posterior to stay near N(0, I)
        kl_loss = -0.5 * self.beta * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        self.add_loss(K.mean(kl_loss))  # add as a regularization loss

        return reconstructed

# Pack encoder to output (μ, logσ^2, z)
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

# Build VAE with a slightly stronger KL weight
vae = VAE(encoder, decoder, beta=0.2)

# Compile with reconstruction loss (MSE); KL is added via add_loss()
vae.compile(optimizer=Adam(learning_rate=0.001), loss=mse)

# Ensure arrays are float32 and shaped (num_samples, 3)
X_train = np.asarray(X_train).astype('float32').reshape(-1, 3)
X_valid = np.asarray(X_valid).astype('float32').reshape(-1, 3)

# Stop early if validation loss stops improving
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train VAE to reconstruct inputs
history = vae.fit(
    X_train, X_train,
    epochs=80,
    batch_size=32,
    validation_data=(X_valid, X_valid),
    callbacks=[early_stopping],
    verbose=1
)

# Plot training curves (loss = reconstruction + KL)
plt.figure(figsize=(8, 5), dpi=300)
plt.plot(history.history['loss'], label='Training Loss', linewidth=2.5)
plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2.5)
plt.title('VAE Training History', fontsize=14, weight='bold')
plt.xlabel('Epoch', fontsize=12); plt.ylabel('Loss', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.6); plt.legend(fontsize=10)
plt.tight_layout(); plt.show()


# Section 3: Generate Synthetic Samples & Evaluation

In [ ]:

# Generate # Synthetic Data
num_samples = 25000 # Number of samples Generated
latent_samples = np.random.normal(size=(num_samples, latent_dim))
synthetic_features = np.clip(decoder.predict(latent_samples), 0, None)

# Convert to DataFrame with 3 features
synthetic_data = pd.DataFrame(synthetic_features, columns=["Margin Loss", "SortedList_Second_Element", "SortedList_First_Element"])

# Required features
features = ['Margin Loss', 'SortedList_Second_Element', 'SortedList_First_Element']
data_selected = data[features]

# Apply Min-Max Scaling only to 'Margin Loss'
scaler_margin = MinMaxScaler(feature_range=(0, 1))
data_selected['Margin Loss'] = scaler_margin.fit_transform(data_selected[['Margin Loss']]).ravel().astype('float64')

# Keep logits as original values
data_selected['SortedList_Second_Element'] = data['SortedList_Second_Element']
data_selected['SortedList_First_Element'] = data['SortedList_First_Element']

data_scaled = pd.DataFrame(data_selected, columns=features)

# Split into training and validation sets
X_train, X_valid = train_test_split(data_scaled, test_size=0.2, random_state=42)

# Convert to NumPy arrays for TensorFlow
X_train = np.array(X_train).astype("float32")
X_valid = np.array(X_valid).astype("float32")

# Dataset shapes
print(f"\nTraining set shape: {X_train.shape}")
print(f"Validation set shape: {X_valid.shape}")

# Plotting area
plt.figure(figsize=(18, 10))
synthetic_data["Margin Loss"] = synthetic_features[:, 0]
synthetic_data["SortedList_Second_Element"] = synthetic_features[:, 1]
synthetic_data["SortedList_First_Element"] = synthetic_features[:, 2]

# Save synthetic dataset
synthetic_data.to_csv("Synthetic_Data.csv", index=False)

# Load real data for comparison
real_data = pd.read_csv("")[['Margin Loss', 'SortedList_Second_Element', 'SortedList_First_Element']]     # Insert ImageNet Benign file path

# Redundancy Analysis
num_duplicates = synthetic_data.duplicated().sum()
print(f"Number of duplicate rows in the synthetic dataset: {num_duplicates}")

num_duplicates = real_data.duplicated().sum()
print(f"Number of duplicate rows in the real dataset: {num_duplicates}")

# Count unique values in each feature for synthetic data
unique_synthetic_margin = synthetic_data['Margin Loss'].nunique()
unique_synthetic_sorted2 = synthetic_data['SortedList_Second_Element'].nunique()
unique_synthetic_sorted1 = synthetic_data['SortedList_First_Element'].nunique()

# Count unique values in each feature for real data
unique_real_margin = real_data['Margin Loss'].nunique()
unique_real_sorted2 = real_data['SortedList_Second_Element'].nunique()
unique_real_sorted1 = real_data['SortedList_First_Element'].nunique()

print(f"\nUnique values in 'Margin Loss': Synthetic ({unique_synthetic_margin}) vs Real ({unique_real_margin})")
print(f"Unique values in 'SortedList_Second_Element': Synthetic ({unique_synthetic_sorted2}) vs Real ({unique_real_sorted2})")
print(f"Unique values in 'SortedList_First_Element': Synthetic ({unique_synthetic_sorted1}) vs Real ({unique_real_sorted1})")

# Check for overlap between real and synthetic data
matching_samples = synthetic_data.merge(real_data, on=['Margin Loss', 'SortedList_Second_Element', 'SortedList_First_Element'], how='inner')
num_matching_samples = matching_samples.shape[0]

print(f"\nNumber of exact matches between synthetic and real data: {num_matching_samples}")
matching_percentage = (num_matching_samples / len(synthetic_data)) * 100
print(f"Percentage of synthetic data that exactly matches real data: {matching_percentage:.2f}%")

# Count total unique values in the original benign dataset
unique_real_data_count = real_data.nunique().sum()
unique_synthetic_data_count = synthetic_data.nunique().sum()

print(f"\nTotal unique values in the original benign dataset: {unique_real_data_count}")
print(f"Total unique values in the synthetic dataset: {unique_synthetic_data_count}")

# Plot comparison of real vs. synthetic distributions
fig, axes = plt.subplots(1, 3, figsize=(16, 4), dpi=300)

sns.histplot(real_data['Margin Loss'], kde=True, bins=30, color='blue', label='Real Margin Loss', stat='density', alpha=0.6, ax=axes[0])
sns.histplot(synthetic_data['Margin Loss'], kde=True, bins=30, color='red', label='Synthetic Margin Loss', stat='density', alpha=0.6, ax=axes[0])
axes[0].set_title('Distribution of Margin Loss')
axes[0].legend()

sns.histplot(real_data['SortedList_Second_Element'], kde=True, bins=30, color='blue', label='Real SortedList_Second_Element', stat='density', alpha=0.6, ax=axes[1])
sns.histplot(synthetic_data['SortedList_Second_Element'], kde=True, bins=30, color='red', label='Synthetic SortedList_Second_Element', stat='density', alpha=0.6, ax=axes[1])
axes[1].set_title('Distribution of Second Top Logit')
axes[1].legend()

sns.histplot(real_data['SortedList_First_Element'], kde=True, bins=30, color='blue', label='Real SortedList_First_Element', stat='density', alpha=0.6, ax=axes[2])
sns.histplot(synthetic_data['SortedList_First_Element'], kde=True, bins=30, color='red', label='Synthetic SortedList_First_Element', stat='density', alpha=0.6, ax=axes[2])
axes[2].set_title('Distribution of Top Logit')
axes[2].legend()

plt.tight_layout()
plt.show()

# Compare basic statistics
print("\nComparison of Real vs Synthetic Data Statistics:")
print("\nReal Data Statistics:\n", real_data.describe())
print("\nSynthetic Data Statistics:\n", synthetic_data.describe())

# Display first few synthetic samples
print("\nFirst 5 samples of the synthetic dataset:")
print(synthetic_data.head())